# Roles y responsabilidades en este notebook

- **Data Engineer**: Responsable de la adquisición, integración, almacenamiento y calidad de los datos. Automatiza flujos de datos y asegura la trazabilidad.
- **Data Scientist**: Realiza análisis exploratorio, define variables, valida supuestos y extrae insights del dataset.
- **ML Engineer**: Implementa pipelines, optimiza procesamiento, integra modelos y asegura reproducibilidad.
- **Software Engineer**: Garantiza estructura modular, calidad de código, automatización, integración continua y buenas prácticas de desarrollo.

# **Limpieza de Datos**

Identificar y manejar valores nulos, duplicados, inconsistencias o posibles outliers.

# 1.- **Importar librerias**

In [26]:
# Importar librerías estándar
import sys
import os

# Detectar la raíz del proyecto de forma robusta (buscando 'src' o '.dvc')
def find_project_root(path):
    while path != os.path.dirname(path):
        items = set(os.listdir(path))
        if 'src' in items or '.dvc' in items:
            return path
        path = os.path.dirname(path)
    raise FileNotFoundError("No se encontró la raíz del proyecto (carpeta 'src' o '.dvc').")

# 1) Calcular y exponer la ruta raíz del proyecto
ruta_raiz_proyecto = find_project_root(os.getcwd())
sys.path.append(ruta_raiz_proyecto)

# 2) Importar funciones del proyecto
from src.cargar_analisis import cargar_dataframe, crear_listas_variables
from src.limpieza import limpiar_y_detectar_atipicos, eliminar_atipicos, guardar_dataframe
from src.carga_dvc import run, save_csv_and_push

# 3) Mostrar diagnóstico rápido
print("Funciones importadas exitosamente desde src/")
print(f"Raíz del proyecto detectada: {ruta_raiz_proyecto}")
csv_model_path = os.path.join(ruta_raiz_proyecto, 'data', 'raw', 'obesity_estimation_model.csv')
print(f"CSV esperado: {csv_model_path} | Existe: {os.path.exists(csv_model_path)}")

Funciones importadas exitosamente desde src/
Raíz del proyecto detectada: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityMine53
CSV esperado: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityMine53\data\raw\obesity_estimation_model.csv | Existe: True


# **2.- Importar datos**

In [27]:
# Ruta de la base de datos
path_data = os.path.join(ruta_raiz_proyecto, 'data', 'raw', 'obesity_estimation_model.csv')

# Ejecutar función para cargar datos
df = cargar_dataframe(path_data)

# Ejecutar la función para generar listas con las variables numericas, categoricas y objetivo
variables_numericas, variables_categoricas, variable_objetivo = crear_listas_variables(to_lower=True)

Archivo CSV cargado exitosamente desde: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityMine53\data\raw\obesity_estimation_model.csv


# **3.- Limpieza de datos**

In [28]:
# Paso 1: Crear una copia limpia del DataFrame
df_limpio = df.copy()

# Estandarizar nombres de columnas
df_limpio.columns = (
    df_limpio.columns
      .str.strip()
      .str.replace(' ', '_')
      .str.replace('-', '_')
      .str.lower()
)

# Eliminar duplicados
df_limpio = df_limpio.drop_duplicates()

print("DataFrame limpio creado con éxito")

DataFrame limpio creado con éxito


# **4.- Guardar los Resultados**

In [29]:
# Paso 2: Eliminar los atípicos del dataframe ya limpio
df_final = eliminar_atipicos(df_limpio, age_range=(5, 50), height_max=2.5, ncp_max=10.0, iqr_factor=1.5)
df_final.head()

---

## **Proceso de Eliminación de Atípicos**

---

1. Aplicando reglas de negocio para 'age', 'height' y 'ncp'...
   - Se eliminaron 0 filas con reglas de negocio.

2. Aplicando método IQR para el resto de variables numéricas...
   - Se eliminaron 0 filas adicionales con el método IQR.


### Reporte Final de Eliminación

Filas iniciales: 1,909
Filas finales:   1,909
Total de filas eliminadas: 0 (0.00%)


---


Proceso de eliminación de atípicos finalizado.


,gender,age,height,weight,family_history_with_overweight,favc,fcvc,ncp,caec,smoke,ch2o,scc,faf,tue,calc,mtrans,nobeyesdad,imc
0,female,21.0,1.62,64.0,yes,no,2.0,3.0,sometimes,no,2.0,no,0.0,1.0,no,public_transportation,1,24.386526
1,female,21.0,1.52,56.0,yes,no,3.0,3.0,sometimes,yes,3.0,yes,3.0,0.0,sometimes,public_transportation,1,24.238227
2,male,23.0,1.80,77.0,yes,no,2.0,3.0,sometimes,no,2.0,no,2.0,1.0,frequently,public_transportation,1,23.765432
3,male,27.0,1.80,87.0,no,no,3.0,3.0,sometimes,no,2.0,no,2.0,0.0,frequently,walking,2,26.851852
4,male,22.0,1.78,89.8,no,no,2.0,1.0,sometimes,no,2.0,no,0.0,0.0,sometimes,public_transportation,3,28.342381


In [30]:
# Llamar a la función para guardar resultados
import os
nombre_archivo = 'obesity_estimation_cleaned.csv'
ruta_destino = os.path.join(ruta_raiz_proyecto, 'data','raw')

# Crear la carpeta si no existe
os.makedirs(ruta_destino, exist_ok=True)

output_path = os.path.join(ruta_destino, nombre_archivo)
guardado_exitoso = False
try:
    df_final.to_csv(output_path, index=False)
    guardado_exitoso = True
    print(f"\nArchivo guardado exitosamente en: {output_path}")
except Exception as e:
    print(f"\nError al guardar el archivo: {e}")

# Resultado
if guardado_exitoso:
    print("\nProceso de guardado finalizado con éxito.")
else:
    print("\nProceso de guardado fallido. Revisar la ruta destino.")


Archivo guardado exitosamente en: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityMine53\data\raw\obesity_estimation_cleaned.csv

Proceso de guardado finalizado con éxito.


# **5.- Guardar los Resultados en DVC**

In [31]:
# Guardar dataset limpio y actualizar DVC/Git
try:
    # Detectar la raíz del proyecto buscando la carpeta que contiene 'src'
    def find_project_root(path):
        while path != os.path.dirname(path):
            if 'src' in os.listdir(path):
                return path
            path = os.path.dirname(path)
        raise FileNotFoundError("No se encontró la carpeta 'src' en la jerarquía de carpetas.")

    project_root = find_project_root(os.getcwd())
    print(f"Cambiando al directorio: {project_root}")
    os.chdir(project_root)

    # Guardar el CSV en data/raw/
    output_path = os.path.join(project_root, "data", "raw", "obesity_estimation_cleaned.csv")
    df_final.to_csv(output_path, index=False)
    print(f"CSV guardado localmente en: {output_path}")

    # Comandos DVC y Git
    from src.carga_dvc import run
    import yaml
    import os
    import glob

    # Configurar Git
    try:
        run('git config user.name "JCGarcesDC"')
        run('git config user.email "A01796283@tec.mx"')
    except Exception as e:
        print(f"Error config git: {e}")

    # Asegurar que estamos en la rama correcta
    try:
        run('git checkout dev2')
    except Exception as e:
        run('git checkout -b dev2')

    # Escribir .gitignore correcto para permitir archivos .dvc dentro de data/
    with open('.gitignore', 'w', encoding='utf-8') as f:
        f.write("""
# Python
__pycache__/
*.py[cod]
*$py.class

.Python
build/
dist/

# Jupyter
.ipynb_checkpoints/
*/.ipynb_checkpoints/*

# Environments
.env
.venv
env/
venv/
ENV/

.gitignore
.gitattributes

# MLflow
mlruns/
mlartifacts/
outputs/

# Data (ignorar datos, mantener metadatos de DVC)
data/**
!data/**/*.dvc
!data/**/.gitkeep
!data/**/*.gitignore
!data/README.md

# Models
models/*
!models/.gitkeep

# Credentials
/config/credentials.yaml
.dvc/*.json

# DVC
.dvc/
/dvc.lock
/dvc.yaml

# Logs
*.log
logs/
""")

    # Inicializar DVC si es necesario
    if not os.path.exists('.dvc'):
        run('dvc init')

    # Configurar remoto DVC usando Google Cloud Storage
    # Buscar archivo de credenciales de GCS en .dvc/
    gcs_cred_files = glob.glob(os.path.join(project_root, '.dvc', '*mna-mlops*.json'))
    if gcs_cred_files:
        gcs_cred_path = gcs_cred_files[0]
        print(f"Credenciales GCS encontradas: {os.path.basename(gcs_cred_path)}")
        
        # Configurar remoto GCS con el bucket correcto
        gcs_bucket = "gs://obesityestimation-mna-mlops-53"
        run(f'dvc remote add -f -d storage {gcs_bucket}')
        run(f'dvc remote modify storage credentialpath "{gcs_cred_path}"')
        print(f"Remoto DVC configurado: {gcs_bucket}")
    else:
        # Fallback: usar directorio local si no hay credenciales GCS
        print("No se encontraron credenciales GCS. Usando remoto local.")
        remote_dir = os.path.join(project_root, 'dvc_remote')
        os.makedirs(remote_dir, exist_ok=True)
        run(f'dvc remote add -f -d storage "{remote_dir}"')

    # Asegurar que no existe un .dvc previo (no crítico si falla)
    try:
        run('dvc remove data/raw/obesity_estimation_cleaned.csv.dvc')
    except Exception as e:
        print(f"Aviso remove dvc file (puede ser normal): {e}")

    # Añadir el archivo a DVC
    run('dvc add -f data/raw/obesity_estimation_cleaned.csv')
    print("Archivo añadido a DVC.")

    # Stage changes
    run('git add -f .gitignore')
    run('git add -f data/raw/obesity_estimation_cleaned.csv.dvc')
    run('git add -f .dvc/config')

    # Commit idempotente: si no hay cambios, omite el commit sin error
    try:
        run('git commit -m "Track cleaned dataset with DVC and configure GCS remote"')
        print("Cambios commiteados en Git.")
    except Exception:
        print("No hay cambios para commitear; se omite el commit.")

    # --- Push seguro a GitHub leyendo token desde config/credentials.yaml o variable de entorno ---
    # Prioridad: variable de entorno GITHUB_TOKEN > archivo config/credentials.yaml > sin push automático
    github_token = os.getenv("GITHUB_TOKEN")
    if not github_token:
        cred_path = os.path.join(project_root, 'config', 'credentials.yaml')
        if os.path.exists(cred_path):
            with open(cred_path, 'r', encoding='utf-8') as f:
                creds = yaml.safe_load(f) or {}
            github_token = ((creds.get('github') or {}).get('token'))

    if github_token and github_token != 'REPLACE_WITH_YOUR_TOKEN':
        print("GITHUB_TOKEN disponible. Actualizando remote temporalmente para push...")
        run(f'git remote set-url origin https://{github_token}@github.com/JCGarcesDC/ObesityEstimation53.git')
        print("Ejecuta manualmente ahora: git push origin dev2")
        print("Luego restaura la URL remota sin token (recomendado):")
        print("git remote set-url origin https://github.com/JCGarcesDC/ObesityEstimation53.git")
    else:
        print("No se encontró un token válido. Configure GITHUB_TOKEN o edite config/credentials.yaml.")
        print("Ejemplos:")
        print("PowerShell (temporal): $env:GITHUB_TOKEN=\"tu_token\"")
        print("Bash (temporal): export GITHUB_TOKEN=tu_token")
        print("Luego ejecute: git push origin dev2")

    # Push de artefactos de DVC (usará el remoto GCS o local según configuración)
    print("\nPara subir datos a DVC ejecute: dvc push")
    print("Esto subirá los datos al bucket de Google Cloud Storage configurado.")

except Exception as e:
    print(f"Error: {str(e)}")
    print("\nEstado actual:")
    print(f"Directorio actual: {os.getcwd()}")
    if os.path.exists('.dvc'):
        print("El directorio .dvc existe")
        print("Contenido de .dvc:")
        print(os.listdir('.dvc'))
        print("\nContenido de .dvc/config:")
        with open('.dvc/config', 'r', encoding='utf-8') as f:
            print(f.read())

Cambiando al directorio: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityMine53
CSV guardado localmente en: d:\OneDrive\Escritorio\Maestria IA\Trimestre 4\MLOps\Git_Local\ObesityMine53\data\raw\obesity_estimation_cleaned.csv
Credenciales GCS encontradas: obesityestimation-mna-mlops-53-5118ae1196dd.json
Credenciales GCS encontradas: obesityestimation-mna-mlops-53-5118ae1196dd.json
Remoto DVC configurado: gs://obesityestimation-mna-mlops-53
Remoto DVC configurado: gs://obesityestimation-mna-mlops-53
Archivo añadido a DVC.
Archivo añadido a DVC.
Cambios commiteados en Git.
No se encontró un token válido. Configure GITHUB_TOKEN o edite config/credentials.yaml.
Ejemplos:
PowerShell (temporal): $env:GITHUB_TOKEN="tu_token"
Bash (temporal): export GITHUB_TOKEN=tu_token
Luego ejecute: git push origin dev2

Para subir datos a DVC ejecute: dvc push
Esto subirá los datos al bucket de Google Cloud Storage configurado.
Cambios commiteados en Git.
No se encontró un token válido

In [32]:
os.system("dvc push")

255